# Standard LVGP regression models

In this notebook, we will demonstrate training and analyzing standard LVGP models using the Branin function.

In [11]:
import torch
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import gpytorch

from lvgp_pytorch.models import LVGPR
from lvgp_pytorch.optim import fit_model_scipy
from lvgp_pytorch.optim.mll_noise_tune import noise_tune
from lvgp_pytorch.utils.variables import NumericalVariable,CategoricalVariable,IntegerVariable
from lvgp_pytorch.utils.input_space import InputSpace

from typing import Dict
from copy import deepcopy
import timeit

%matplotlib inline
plt.rcParams['figure.dpi']=150
plt.rcParams['font.family']='serif'

In [12]:
# configuration space
config = InputSpace()
x = NumericalVariable(name='x',lower=-5,upper=10)
t = CategoricalVariable(name='t',levels=np.linspace(1,4,4))
config.add_inputs([x,t])

config

Input space with variables:

x, Type: Numerical, Range: [-5.0,10.0]
t, Type: Categorical, Levels: {1.0, 2.0, 3.0, 4.0}

In [13]:
def branin(params:Dict)->float:
    a, b, c, r, s, t1 = 1, 5.1/4/(np.pi**2), 5/np.pi, 6, 10, 1/8/np.pi
    x = params["x"]
    t = (params["t"]-1)*5.0
    return a * (t - b * x**2 + c*x - r)**2 + s*(1-t1)*np.cos(x) + s

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

In [14]:
# generate 100 samples
set_seed(1)
num_samples = 20
train_x = torch.from_numpy(
    config.random_sample(np.random,num_samples)
)
train_y = [None]*num_samples

for i,x in enumerate(train_x):
    train_y[i] = branin(config.get_dict_from_array(x.numpy()))

train_y = torch.tensor(train_y).double()


# generate 1000 test samples
num_samples = 1000
test_x = torch.from_numpy(config.random_sample(np.random,num_samples))
test_y = [None]*num_samples

for i,x in enumerate(test_x):
    test_y[i] = branin(config.get_dict_from_array(x.numpy()))
    
# create tensor objects
test_y = torch.tensor(test_y).to(train_y)

In [15]:
train_x

tensor([[4.1702e-01, 1.0000e+00],
        [7.2032e-01, 1.0000e+00],
        [1.1437e-04, 1.0000e+00],
        [3.0233e-01, 3.0000e+00],
        [1.4676e-01, 3.0000e+00],
        [9.2339e-02, 1.0000e+00],
        [1.8626e-01, 2.0000e+00],
        [3.4556e-01, 1.0000e+00],
        [3.9677e-01, 1.0000e+00],
        [5.3882e-01, 0.0000e+00],
        [4.1919e-01, 0.0000e+00],
        [6.8522e-01, 1.0000e+00],
        [2.0445e-01, 0.0000e+00],
        [8.7812e-01, 0.0000e+00],
        [2.7388e-02, 1.0000e+00],
        [6.7047e-01, 3.0000e+00],
        [4.1730e-01, 3.0000e+00],
        [5.5869e-01, 2.0000e+00],
        [1.4039e-01, 1.0000e+00],
        [1.9810e-01, 0.0000e+00]], dtype=torch.float64)

## Creating a LVGP instance

In [16]:
'''
from LVGP_MATLAB_connector import LVGP_MATLAB
train_xm, train_ym, test_xm, test_ym = train_x.numpy(), train_y.numpy(), test_x.numpy(), test_y.numpy()

start = timeit.default_timer()
model_m = LVGP_MATLAB()
model_m.fit(train_xm, train_ym[:, np.newaxis], ind_qual=config.qual_index)
test_mean, test_std = model_m.predict(test_xm)
stop = timeit.default_timer()
rrmse = np.sqrt(np.mean((test_ym[:,np.newaxis]-test_mean)**2))/np.std(test_ym)
print('RRMSE: %5.3f'%rrmse.item())
print('Fit time: ', stop - start)
'''

"\nfrom LVGP_MATLAB_connector import LVGP_MATLAB\ntrain_xm, train_ym, test_xm, test_ym = train_x.numpy(), train_y.numpy(), test_x.numpy(), test_y.numpy()\n\nstart = timeit.default_timer()\nmodel_m = LVGP_MATLAB()\nmodel_m.fit(train_xm, train_ym[:, np.newaxis], ind_qual=config.qual_index)\ntest_mean, test_std = model_m.predict(test_xm)\nstop = timeit.default_timer()\nrrmse = np.sqrt(np.mean((test_ym[:,np.newaxis]-test_mean)**2))/np.std(test_ym)\nprint('RRMSE: %5.3f'%rrmse.item())\nprint('Fit time: ', stop - start)\n"

In [17]:
# create LVGP instance
set_seed(4)
start = timeit.default_timer()
model = LVGPR(
    train_x=train_x,
    train_y=train_y,
    qual_index=config.qual_index,
    quant_index=config.quant_index,
    num_levels_per_var=list(config.num_levels.values()),
    quant_correlation_class="RBFKernel",
    noise=1, 
    fix_noise=False,
).double()

## Optimization using multiple random starts

In [18]:
# fit model with 50 different starts
reslist,nll_inc = fit_model_scipy(
    model,
    num_restarts=49, # number of starting points
    options={'ftol':1e-6} # options to L-BFGS
)

# set model to eval model; default is in train model
_ = model.eval()
stop = timeit.default_timer()

In [19]:
# prediction on test set
with torch.no_grad():
    # set return_std = False if standard deviation is not needed 
    test_mean,test_std = model.predict(test_x,return_std=True)
    
# print RRMSE
rrmse = torch.mean((test_y-test_mean)**2).sqrt()/test_y.std()
print('RRMSE : %5.3f'%rrmse.item())
print('Fit time: ', stop - start)

RRMSE : 0.160
Fit time:  37.0511578


In [20]:

'''
# plot latent values
n_fig = np.shape(config.qual_index)[0]
fig,axs = plt.subplots(1,n_fig,figsize=(10,4))

for i in range(n_fig):
    latents = model.lv_mapping_layers[i].latents.detach().numpy()
    _ = axs[i].plot(latents[:,0],latents[:,1],'k.')
    
    hyp = config.get_variable_by_idx(config.qual_index[i])
    # annotate the labels
    for j,level in enumerate(hyp.levels):
        _ = axs[i].annotate(
            str(level),latents[j,:],
            textcoords = 'offset points',
            xytext = (-1,3),
            size='8'
        )
        
    
    _ = axs[i].set_xlabel(r'$z_1$')
    _ = axs[i].set_ylabel(r'$z_2$')
    _ = axs[i].set_title(r'$%s$' %hyp.name)
    _ = axs[i].grid(alpha=0.5)
    _ = axs[i].set_aspect('equal', 'datalim')

fig.tight_layout()
'''

"\n# plot latent values\nn_fig = np.shape(config.qual_index)[0]\nfig,axs = plt.subplots(1,n_fig,figsize=(10,4))\n\nfor i in range(n_fig):\n    latents = model.lv_mapping_layers[i].latents.detach().numpy()\n    _ = axs[i].plot(latents[:,0],latents[:,1],'k.')\n    \n    hyp = config.get_variable_by_idx(config.qual_index[i])\n    # annotate the labels\n    for j,level in enumerate(hyp.levels):\n        _ = axs[i].annotate(\n            str(level),latents[j,:],\n            textcoords = 'offset points',\n            xytext = (-1,3),\n            size='8'\n        )\n        \n    \n    _ = axs[i].set_xlabel(r'$z_1$')\n    _ = axs[i].set_ylabel(r'$z_2$')\n    _ = axs[i].set_title(r'$%s$' %hyp.name)\n    _ = axs[i].grid(alpha=0.5)\n    _ = axs[i].set_aspect('equal', 'datalim')\n\nfig.tight_layout()\n"

## An improved optimization method

In [21]:
set_seed(4)
start = timeit.default_timer()

model2 = LVGPR(
    train_x=train_x,
    train_y=train_y,
    qual_index=config.qual_index,
    quant_index=config.quant_index,
    num_levels_per_var=list(config.num_levels.values()),
    quant_correlation_class="RBFKernel",
    noise=1, 
    fix_noise=False
).double()

# optimize noise successively
nll_inc_tuned,opt_history = noise_tune(
    model2,
    num_restarts=19, # num of starting points at the largest noise variance
    options={'ftol':1e-8}
)
stop = timeit.default_timer()
# 
print('NLL obtained from multi-start optimization....: %6.2f'%nll_inc)
print('NLL obtained from noise tuning strategy.......: %6.2f'%nll_inc_tuned)

NLL obtained from multi-start optimization....:   1.13
NLL obtained from noise tuning strategy.......:   1.13


In [22]:
# prediction on test set
with torch.no_grad():
    # set return_std = False if standard deviation is not needed
    # set include_noise = True, if noise variance is to be included
    # in the posterior variance 
    test_mean2,test_std2 = model2.predict(test_x,return_std=True)
    

# print RRMSE
rrmse = torch.mean((test_y-test_mean2)**2).sqrt()/test_y.std()
print('RRMSE : %5.3f'%rrmse.item())
print('Fit time: ', stop - start)

RRMSE : 0.153
Fit time:  8.496377300000006


In [23]:
'''
# plot latent values
fig,axs = plt.subplots(1,n_fig,figsize=(10,4))

for i in range(n_fig):
    latents = model2.lv_mapping_layers[i].latents.detach().numpy()
    _ = axs[i].plot(latents[:,0],latents[:,1],'k.')
    
    hyp = config.get_variable_by_idx(config.qual_index[i])
    # annotate the labels
    for j,level in enumerate(hyp.levels):
        _ = axs[i].annotate(
            str(level),latents[j,:],
            textcoords = 'offset points',
            xytext = (-1,3),
            size='8'
        )
        
    
    _ = axs[i].set_xlabel(r'$z_1$')
    _ = axs[i].set_ylabel(r'$z_2$')
    _ = axs[i].set_title(r'$%s$' %hyp.name)
    _ = axs[i].grid(alpha=0.5)
    _ = axs[i].set_aspect('equal', 'datalim')

fig.tight_layout()
'''

"\n# plot latent values\nfig,axs = plt.subplots(1,n_fig,figsize=(10,4))\n\nfor i in range(n_fig):\n    latents = model2.lv_mapping_layers[i].latents.detach().numpy()\n    _ = axs[i].plot(latents[:,0],latents[:,1],'k.')\n    \n    hyp = config.get_variable_by_idx(config.qual_index[i])\n    # annotate the labels\n    for j,level in enumerate(hyp.levels):\n        _ = axs[i].annotate(\n            str(level),latents[j,:],\n            textcoords = 'offset points',\n            xytext = (-1,3),\n            size='8'\n        )\n        \n    \n    _ = axs[i].set_xlabel(r'$z_1$')\n    _ = axs[i].set_ylabel(r'$z_2$')\n    _ = axs[i].set_title(r'$%s$' %hyp.name)\n    _ = axs[i].grid(alpha=0.5)\n    _ = axs[i].set_aspect('equal', 'datalim')\n\nfig.tight_layout()\n"